In [1]:
!pip install kaggle

In [2]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets list

ref                                                    title                                        size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------  ------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
thedrcat/daigt-v2-train-dataset                        DAIGT V2 Train Dataset                       29MB  2023-11-16 01:38:36           2873        243  1.0              
thedevastator/global-video-game-sales-and-reviews      Global Video Game Sales and Reviews          56KB  2023-12-20 06:36:51           1020         29  1.0              
bhavikjikadara/car-price-prediction-dataset            Car Price Prediction Dataset                  4KB  2023-12-23 07:07:54           1071         31  0.9411765        
muhammadbinimran/housing-price-prediction-data         Housing Price Prediction Data               763KB  2023-11-21 17:56:32          12364     

In [7]:
!kaggle datasets download -d arbethi/diabetic-retinopathy-level-detection

100% 9.64G/9.66G [01:35<00:00, 144MB/s]
100% 9.66G/9.66G [01:35<00:00, 108MB/s]


In [8]:
!unzip -q /content/diabetic-retinopathy-level-detection.zip

In [9]:
imageSize = [299, 299]
trainPath = r'/content/preprocessed dataset/preprocessed dataset/train'
testPath = r'/content/preprocessed dataset/preprocessed dataset/test'

In [10]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.xception import Xception, preprocess_input
from glob import glob
import matplotlib.pyplot as plt
import numpy as np

In [11]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
training_set = train_datagen.flow_from_directory('/content/preprocessed dataset/preprocessed dataset/training',
                                      target_size=(299, 299),
                                      class_mode='categorical',
                                      batch_size=32)
test_set = test_datagen.flow_from_directory('/content/preprocessed dataset/preprocessed dataset/testing',
                                    target_size=(299, 299),
                                      class_mode='categorical',
                                      batch_size=32)

Found 3662 images belonging to 5 classes.
Found 734 images belonging to 5 classes.


In [13]:
training_set.class_indices

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4}

In [14]:
xception = Xception(input_shape=imageSize + [3], weights='imagenet', include_top=False)

83683744/83683744 [==============================] - 1s 0us/step


In [15]:
for layer in xception.layers:
  layer.trainable = False

In [16]:
x = Flatten()(xception.output)

In [17]:
prediction = Dense(5, activation='softmax')(x)

In [18]:
model = Model(inputs=xception.input, outputs=prediction)

In [19]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [20]:
batch_size = 32
steps_per_epoch = len(training_set) // batch_size

In [22]:
model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set) // batch_size,

  validation_steps=len(test_set)
)

<ipython-input-22-d1e09f269fa2>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
3/3 [==============================] - 440s 208s/step - loss: 6.6931 - accuracy: 0.5833 - val_loss: 7.8143 - val_accuracy: 0.4005
Epoch 2/10
3/3 [==============================] - 412s 195s/step - loss: 7.8557 - accuracy: 0.3958 - val_loss: 7.8848 - val_accuracy: 0.5940
Epoch 3/10
3/3 [==============================] - 439s 207s/step - loss: 6.2438 - accuracy: 0.6875 - val_loss: 7.2710 - val_accuracy: 0.6921
Epoch 4/10
3/3 [==============================] - 440s 208s/step - loss: 6.6104 - accuracy: 0.6875 - val_loss: 8.1657 - val_accuracy: 0.5490
Epoch 5/10
3/3 [==============================] - 424s 201s/step - loss: 6.7226 - accuracy: 0.5000 - val_loss: 5.6427 - val_accuracy: 0.5831
Epoch 6/10
3/3 [==============================] - 444s 210s/step - loss: 4.7296 - accuracy: 0.5938 - val_loss: 4.5489 - val_accuracy: 0.6540
Epoch 7/10
3/3 [==============================] - 439s 208s/step - loss: 5.8217 - accuracy: 0.6042 - val_loss: 5.1287 - val_accuracy: 0.6880
Epoch 8/10
3/

In [23]:
model.save('Colab_Model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
